In [1]:
import pandas as pd
import os

In [82]:
df = pd.read_csv("temp_data.csv")

In [83]:
print(df.shape)
print(df.isna().sum())
print(df.columns)

(1235, 27)
SentAt             0
StartedAt        691
Control          691
Sad              691
Intrusion        691
Encourage        691
FunThings        691
IgotThis         691
Company          691
Alone            691
Thoughts         691
Safe             691
Appointments     691
Cancel           691
Laying           691
Sleep            691
Qualitysleep    1072
Out              691
Eaten            691
Avoid            691
Useful           691
Enjoy            691
Pleas            691
Pleasant        1076
Unpl             691
Unpleasant      1211
Remarks         1087
dtype: int64
Index(['SentAt', 'StartedAt', 'Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Qualitysleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Pleasant', 'Unpl',
       'Unpleasant', 'Remarks'],
      dtype='object')


In [84]:
df = df.drop(columns = ['Qualitysleep', 'Pleasant', 'Unpleasant', 'Remarks'])

In [87]:
#drop all which have all missing except for SentAt
to_drop =  ['StartedAt', 'Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Unpl']

df = df.dropna(subset = to_drop, how = 'any', axis=0)

In [91]:
print(df.shape)
print(df.isna().sum().sum())
print(df.columns)

(544, 23)
0
Index(['SentAt', 'StartedAt', 'Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Out', 'Eaten', 'Avoid',
       'Useful ', 'Enjoy', 'Pleas', 'Unpl'],
      dtype='object')


In [95]:
df['SentAt']
#first = 16/03/2017
#last = 27/04/2018
# just over a year in scope
# note that we have 544 obs but not evenly spaces out

3       16/03/2017 15:30
5       17/03/2017 09:00
7       17/03/2017 21:00
8       18/03/2017 09:00
9       18/03/2017 15:00
              ...       
1198    18/04/2018 22:00
1203    20/04/2018 16:00
1206    21/04/2018 16:00
1219    25/04/2018 22:00
1224    27/04/2018 16:00
Name: SentAt, Length: 544, dtype: object

In [96]:
df.head()

,SentAt,StartedAt,Control,Sad,Intrusion,Encourage,FunThings,IgotThis,Company,Alone,...,Cancel,Laying,Sleep,Out,Eaten,Avoid,Useful,Enjoy,Pleas,Unpl
3,16/03/2017 15:30,16/03/2017 15:30,70.00,42.02,76.74,59.38,35.54,81.14,0.0,24.92,...,1.0,70.78,0.0,66.63,61.97,20.52,41.50,79.85,1.0,1.0
5,17/03/2017 09:00,17/03/2017 09:17,17.53,18.74,16.93,81.39,82.59,82.19,1.0,16.93,...,0.0,96.15,1.0,87.41,69.84,2.17,87.11,77.77,1.0,0.0
7,17/03/2017 21:00,17/03/2017 21:05,26.87,27.07,30.18,69.34,54.88,45.44,1.0,19.64,...,0.0,2.97,0.0,62.31,31.39,2.17,50.96,54.48,1.0,1.0
8,18/03/2017 09:00,18/03/2017 09:11,4.78,2.27,10.00,88.62,77.77,79.28,1.0,0.00,...,0.0,98.66,1.0,1.97,51.16,2.87,22.05,70.84,1.0,0.0
9,18/03/2017 15:00,18/03/2017 15:07,1.46,0.00,2.47,82.89,82.69,82.79,1.0,3.07,...,1.0,1.97,0.0,84.00,82.69,1.27,59.60,84.30,1.0,0.0
